### Solving a System of Linear Equations

In this project, we compare the performance of three different methods of solving a system of linear equations. 
 

#### 1. Create a function randmat(n) which returns a random square matrix constructed as the following recipe. 
<ul>
    <li>The size of the matrix is $n \times n$.</li>
    <li>Each off-diagonal entry ($a_{ij}$ where $i \ne j$) is a random number in $[0, 1)$. A random number can be constructed by the random method (see <a href="https://docs.scipy.org/doc/numpy/reference/routines.random.html">here</a>).</li>
    <li>A diagonal entry $a_{ii}$ is a random number in $[n, n+1)$. (This condition guarantees that the matrix $(a_{ij})$ is strictly diagonally dominant, hence invertible.)</li>
</ul>

#### And create a function randvec(n) which returns an $n$-dimensional random vector whose entries are random numbers in $[0, 100)$. 

In [ ]:
import numpy as np
def randmat(n):
    matrix = np.random.rand(n,n)
    for i in range (0,n):
        for j in range(0, n):
            if i == j:
                matrix[i][j] = n + matrix[i][j]
    return matrix

def randvec(n):
    vector = 100*np.random.rand(n)
    return vector

def main():
    matrix = randmat(5)
    vector = randvec(5)
    print(matrix)
    print(vector)
    
main()


#### 2. Create a function GaussElim(A, b) which solves a system of linear equations $Ax = b$ by using Gaussian Elimination with the partial pivoting.

In [ ]:
import numpy as np

def AugmentMatrix(A,b):
    if (len(A)==len(b)):
        augmatrix = np.zeros((len(A),len(b)+1))
        for i in range(0, len(A)):
            for j in range(0, len(b)+1):
                if j < len(b):
                    augmatrix[i][j] = A[i][j]
        for i in range(0, len(b)):
            augmatrix[i][len(b)] = b[i]
        return augmatrix

def AugmentMatrixInverse(A):
    augmatrix = np.zeros((len(A),2*len(A)))
    for i in range(0, len(A)):
        for j in range(0, 2*len(A)):
            if j < len(A) and i < len(A):
                augmatrix[i][j] = A[i][j]
    for i in range(0, len(A)):
        for j in range(len(A), 2*len(A)):
            if(j-len(A)==i):
                augmatrix[i][j] = 1
            else:
                augmatrix[i][j] = 0
    return augmatrix

def randmat(n):
    matrix = np.random.rand(n,n)
    for i in range (0,n):
        for j in range(0, n):
            if i == j:
                matrix[i][j] = n + matrix[i][j]
    return matrix

def randvec(n):
    vector = 100*np.random.rand(n)
    return vector

def MultiplyMatrixVector(A,b):
    if (len(A[0])==len(b)):
        product = np.zeros((len(A)))
        for i in range(0, len(A)):
            for j in range(0, len(b)):
                product[i]= product[i]+A[i][j]*b[j]
    return product

def GaussElim(A,b):
    if len(A) == len(b):
        newA = np.column_stack((A,b))
        n = len(A)
        NROW = np.zeros(n)
        pfound = False
        for i in range(0,n):
            NROW[i] = i
        for i in range(0, n-1):
            p=i
            maxim = 0
            for j in range(i, n):
                if(pfound == False):
                    if(np.absolute(newA[int(NROW[j])][i])>maxim):
                        maxim = np.absolute(newA[int(NROW[j])][i])
                    if(np.absolute(newA[int(NROW[p])][i])==maxim):
                        p = j
                        pfound = True
            if(newA[int(NROW[p])][i]==0):
                print('no unique solution exists')

            if(NROW[i] != NROW[p]):
                temp = NROW[i]
                NROW[i] = NROW[p]
                NROW[p] = temp
            m = np.zeros((n,n+1))
            for j in range(i+1, n):
                m[int(NROW[j])][i]=newA[int(NROW[j])][i]/newA[int(NROW[i])][i]
                newA[int(NROW[j])]=newA[int(NROW[j])]- (m[int(NROW[j])][i]*newA[int(NROW[i])])
      #  print(newA)
        if(newA[int(NROW[n-1])][n-1]==0):
            print('no unique solution exists')

        x = np.zeros(n)
        x[n-1] = newA[int(NROW[n-1])][n]/newA[int(NROW[n-1])][n-1]
        for i in reversed(range(0,n-1)):
            differand = 0
            for j in range(i+1, n):
                differand = differand + newA[int(NROW[i])][j]*x[j]
            x[i] = (newA[int(NROW[i])][n] - differand)/newA[int(NROW[i])][i]    
        return x
    else:
        print('invalid system')

def main():
    A = randmat(3)
    #A=np.array([[3,2],[2,1],[9,8]])
    print('A matrix\n', A)
    #print(AugmentMatrixInverse(A))    
    b = randvec(3)
    print('\nb vector\n', b)
    GaussElim(A,b)
    if(len(A) == len(b)):
        print('\nx vector that solves Ax = b system\n', GaussElim(A,b))
        print('\nmultiply matrix A by vector x and see if the product is b\n', MultiplyMatrixVector(A,GaussElim(A,b)))
main()


#### 3. (10 pts) Create a function Jacobi(A, b, err) which solves a system of linear equations $Ax = b$ by using Jacobi interation method. Set $x^{(0)} = \vec{0}$. We stop the iteration when the estimation of the error $||x^{(k)} - x^{(k-1)}||_\infty$ is less than err or $k = 1000$. (Here $x^{(k)}$ is the $k$-th output of the iteration).

In [ ]:
import numpy as np


def printmatrix(M):
    for i in range(0, len(M)):
        for j in range(0, len(M)):
            print(M[i][j], " ", end="")
        print("")


def MultiplyMatrix(A, B):
    if (len(A[0]) == len(B)):
        product = np.zeros((len(A), len(B[0])))
        for i in range(0, len(A)):
            for j in range(0, len(B[0])):
                for k in range(0, len(B)):
                    product[i][j] = product[i][j] + A[i][k] * B[k][j]
    return product


def AugmentMatrixInverse(A):
    augmatrix = np.zeros((len(A), 2 * len(A)))
    for i in range(0, len(A)):
        for j in range(0, 2 * len(A)):
            if j < len(A) and i < len(A):
                augmatrix[i][j] = A[i][j]
    for i in range(0, len(A)):
        for j in range(len(A), 2 * len(A)):
            if (j - len(A) == i):
                augmatrix[i][j] = 1
            else:
                augmatrix[i][j] = 0
    return augmatrix


def Inverse(A):
    newA = AugmentMatrixInverse(A)
    n = len(newA)
    NROW = np.zeros(n)
    pfound = False
    for i in range(0, n):
        NROW[i] = i
    for i in range(0, n - 1):
        p = i
        maxim = 0
        for j in range(i, n):
            if (pfound == False):
                if (np.absolute(newA[int(NROW[j])][i]) > maxim):
                    maxim = np.absolute(newA[int(NROW[j])][i])
                if (np.absolute(newA[int(NROW[p])][i]) == maxim):
                    p = j
                    pfound = True
        if (newA[int(NROW[p])][i] == 0):
            print('matrix is singular')

        if (NROW[i] != NROW[p]):
            temp = NROW[i]
            NROW[i] = NROW[p]
            NROW[p] = temp
        m = np.zeros((n, n + 1))
        for j in range(i + 1, n):
            m[int(NROW[j])][i] = newA[int(NROW[j])][i] / newA[int(NROW[i])][i]
            newA[int(NROW[j])] = newA[int(NROW[j])] - (m[int(NROW[j])][i] * newA[int(NROW[i])])

    if (newA[int(NROW[n - 1])][n - 1] == 0):
        print('matrix is singular')

    for i in reversed(range(0, n)):
        for j in reversed(range(0, i + 1)):
            # print(i, ' ', j)
            if j == i:
                newA[int(NROW[i])] = newA[int(NROW[i])] / newA[int(NROW[i])][j]
                largesti = int(NROW[i])
                for k in reversed(range(0, i)):
                    newA[int(NROW[k])] = newA[int(NROW[k])] - newA[int(NROW[k])][j] * newA[largesti]
            else:
                newA[int(NROW[i])] = newA[int(NROW[i])] - newA[int(NROW[i])][j] * newA[largesti]
        # print(A1)
    Inverse = np.zeros((n, n))
    for i in range(0, n):
        for j in range(0, n):
            Inverse[i][j] = newA[int(NROW[i])][j + n]
    return Inverse


def MultiplyMatrixVector(A, b):
    if (len(A[0]) == len(b)):
        product = np.zeros(len(A))
        for i in range(0, len(A)):
            for j in range(0, len(b)):
                product[i] = product[i] + A[i][j] * b[j]
    return product


def InfinityNorm(a):
    max = np.absolute(a[0])
    for i in range(0, int(len(a))):
        if np.absolute(a[i]) > max:
            max = np.absolute(a[i])
    return max


def jacobi(A, b, err):
    n = len(A)
    D = np.zeros((n, n))
    L = np.zeros((n, n))
    U = np.zeros((n, n))
    for i in range(0, n):
        for j in range(0, n):
            if (j > i):
                U[i][j] = -1 * A[i][j]
            elif (j < i):
                L[i][j] = -1 * A[i][j]
            else:
                D[i][j] = A[i][j]
    # printmatrix(D)
    # printmatrix(U)
    # printmatrix(L)
    x = np.zeros(n)
    lusum = L + U
    # WE NEED TO WRITE OUR OWN MATRIX INVERSION AND MATRIX MULTIPLICATION
    coeffxk = MultiplyMatrix(Inverse(D), lusum)
    # printmatrix(lusum)
    endarray = MultiplyMatrixVector(Inverse(D), b)
    xprev = np.zeros(n)
    # printmatrix(coeffxk)
    count = 0
    moon = 0
    a = 1
    while (count <= 1000):
        xkterm = MultiplyMatrixVector(coeffxk, x)
        x = xkterm + endarray
        print("iteration ", count + 1, "- x = ", x)
        # print("rotation: ", count, "  x: ", x, "     xprev: ", xprev, "\n")
        a = InfinityNorm(x - xprev)
        print("Error for iteration #", count + 1, ": ", a)
        if (err > a):
            break
        xprev = x
        count += 1
    return x


def main():
    A = np.array([[2,-1,0],[-1,3,-1],[0,-1,2]])
    b = np.array([1,8,-5])
    #A = np.array([[2, -1, 1, 2], [-1, 3, -1, 5], [0, -1, 2, 7], [4, 3, 2, 1]])
    #b = np.array([1, 8, -5, 5])
    err = 0.001
    print(A)
    print('Jacobi Method')
    xkplus1 = jacobi(A, b, err)


main()


#### 4. (10 pts) Create a function GaussSeidel(A, b, err) which solves a system of linear equations $Ax = b$ by using Gauss-Seidel interation method. Set $x^{(0)} = \vec{0}$. We stop the iteration when the estimation of the error $||x^{(k)} - x^{(k-1)}||_\infty$ is less than err or $k = 1000$. (Here $x^{(k)}$ is the $k$-th output of the iteration).

In [ ]:
import numpy as np

def printmatrix(M):
    for i in range (0,len(M)):
        for j in range(0, len(M)):
            print(M[i][j], " ", end = "")
        print("")

        
def MultiplyMatrix(A,B):
    if (len(A[0])==len(B)):
        product = np.zeros((len(A),len(B[0])))
        for i in range(0, len(A)):
            for j in range(0, len(B[0])):
                for k in range(0, len(B)):
                    product[i][j] = product[i][j]+A[i][k]*B[k][j]
    return product

def MultiplyMatrixVector(A,b):
    if (len(A[0])==len(b)):
        product = np.zeros((len(A)))
        for i in range(0, len(A)):
            for j in range(0, len(b)):
                product[i]= product[i]+A[i][j]*b[j]
    return product


def AugmentMatrixInverse(A):
    augmatrix = np.zeros((len(A),2*len(A)))
    for i in range(0, len(A)):
        for j in range(0, 2*len(A)):
            if j < len(A) and i < len(A):
                augmatrix[i][j] = A[i][j]
    for i in range(0, len(A)):
        for j in range(len(A), 2*len(A)):
            if(j-len(A)==i):
                augmatrix[i][j] = 1
            else:
                augmatrix[i][j] = 0
    return augmatrix

def Inverse(A):
    newA = AugmentMatrixInverse(A)
    n = len(newA)
    NROW = np.zeros(n)
    pfound = False
    for i in range(0,n):
        NROW[i] = i
    for i in range(0, n-1):
        p = i
        maxim = 0
        for j in range(i, n):
            if(pfound == False):
                if(np.absolute(newA[int(NROW[j])][i])>maxim):
                    maxim = np.absolute(newA[int(NROW[j])][i])
                if(np.absolute(newA[int(NROW[p])][i])==maxim):
                    p = j
                    pfound = True
        if(newA[int(NROW[p])][i]==0):
            print('matrix is singular')
        
        if(NROW[i] != NROW[p]):
            temp = NROW[i]
            NROW[i] = NROW[p]
            NROW[p] = temp
        m = np.zeros((n,n+1))
        for j in range(i+1, n):
            m[int(NROW[j])][i]=newA[int(NROW[j])][i]/newA[int(NROW[i])][i]
            newA[int(NROW[j])]=newA[int(NROW[j])]- (m[int(NROW[j])][i]*newA[int(NROW[i])])
                     
    if(newA[int(NROW[n-1])][n-1]==0):
        print('matrix is singular')
        
    for i in reversed(range(0,n)):
        for j in reversed(range(0,i+1)):
            #print(i, ' ', j)
            if j == i:
                newA[int(NROW[i])] = newA[int(NROW[i])]/newA[int(NROW[i])][j]
                largesti = int(NROW[i])
                for k in reversed(range(0,i)):
                    newA[int(NROW[k])] = newA[int(NROW[k])] - newA[int(NROW[k])][j]*newA[largesti]
            else:
                newA[int(NROW[i])] = newA[int(NROW[i])] - newA[int(NROW[i])][j]*newA[largesti]
        #print(A1)
    Inverse = np.zeros((n,n))
    for i in range(0,n):
        for j in range(0,n):
            Inverse[i][j] = newA[int(NROW[i])][j+n]
    return Inverse

def InfinityNorm(a):
    max = np.absolute(a[0])
    for i in range(0, int(len(a))):
        if np.absolute(a[i]) > max:
            max = np.absolute(a[i])
    return max


def GaussSeidel(A, b, err):
    n = len(A)
    D = np.zeros((n, n))
    L = np.zeros((n, n))
    U = np.zeros((n, n))
    for i in range (0, n):
        for j in range(0, n):
            if (j > i):
                U[i][j] = -1*A[i][j]
            elif (j < i):
                L[i][j] = -1*A[i][j]
            else:
                D[i][j] = A[i][j]
   # printmatrix(D)
   # printmatrix(U)
   # printmatrix(L)
    x = np.zeros(n)
    dldiff = D - L
    #WE NEED TO WRITE OUR OWN MATRIX INVERSION
    coeffxk = MultiplyMatrix(Inverse(dldiff),U)
     #printmatrix(lusum)
    endarray = MultiplyMatrixVector(Inverse(dldiff), b)
        #printmatrix(coeffxk)
    xprev = np.zeros(n)
    # printmatrix(coeffxk)
    count = 0
    moon = 0
    a = 1
    while(count <= 20 and 0.001< a):
        xkterm = MultiplyMatrixVector(coeffxk, x)
        x = xkterm + endarray
        print("iteration ", count + 1, ": x = ", x)
        # print("rotation: ", count, "  x: ", x, "     xprev: ", xprev, "\n")
        a = InfinityNorm(x - xprev)
        print("Error for iteration #", count+1, ": ", a)
        xprev = x
        count += 1

    return x
    
def main():
    A = np.array([[2,-1,0],[-1,3,-1],[0,-1,2]])
    b = np.array([1,8,-5])
    Err = 0.001
    
    print('Gauss-Seidel Method')
    xkplus1 = GaussSeidel(A,b,err)

        
main()




gauss-seidel method
rotation:  0   x:  [ 0.5         2.83333333 -1.08333333]      xprev:  [0 0 0] 

Error:  2.833333333333333
rotation:  1   x:  [ 1.91666667  2.94444444 -1.02777778]      xprev:  [ 0.5         2.83333333 -1.08333333] 

Error:  1.4166666666666665
rotation:  2   x:  [ 1.97222222  2.98148148 -1.00925926]      xprev:  [ 1.91666667  2.94444444 -1.02777778] 

Error:  0.05555555555555558
rotation:  3   x:  [ 1.99074074  2.99382716 -1.00308642]      xprev:  [ 1.97222222  2.98148148 -1.00925926] 

Error:  0.01851851851851838
rotation:  4   x:  [ 1.99691358  2.99794239 -1.00102881]      xprev:  [ 1.99074074  2.99382716 -1.00308642] 

Error:  0.006172839506172867
rotation:  5   x:  [ 1.99897119  2.99931413 -1.00034294]      xprev:  [ 1.99691358  2.99794239 -1.00102881] 

Error:  0.002057613168724215
rotation:  6   x:  [ 1.99965706  2.99977138 -1.00011431]      xprev:  [ 1.99897119  2.99931413 -1.00034294] 

Error:  0.000685871056241405
[ 1.99965706  2.99977138 -1.00011431]


#### 5. For $n = 100, 200, 300, \cdots , 1000$, create a random $n \times n$ matrix $A$ and a random $n$-dimensional vector $b$. Solve the system of linear equations $Ax = b$ by using GaussElim(A, b), Jacobi(A, b, err), and GaussSeidel(A, b, err). Use $10^{-6}$ for the error tolerance. Record the excution time for each method. Plot the graph of the excution time for those three methods on the same plane.

For the computation of the excution time, you may use the following method:

In [ ]:
import time

start = time.time()
"the code you want to test stays here"
end = time.time()

print(end - start)

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt

def MultiplyMatrix(A,B):
    if (len(A[0])==len(B)):
        product = np.zeros((len(A),len(B[0])))
        for i in range(0, len(A)):
            for j in range(0, len(B[0])):
                for k in range(0, len(B)):
                    product[i][j] = product[i][j]+A[i][k]*B[k][j]
    return product

def MultiplyMatrixVector(A,b):
    if (len(A[0])==len(b)):
        product = np.zeros((len(A)))
        for i in range(0, len(A)):
            for j in range(0, len(b)):
                product[i]= product[i]+A[i][j]*b[j]
    return product


def AugmentMatrixInverse(A):
    augmatrix = np.zeros((len(A),2*len(A)))
    for i in range(0, len(A)):
        for j in range(0, 2*len(A)):
            if j < len(A) and i < len(A):
                augmatrix[i][j] = A[i][j]
    for i in range(0, len(A)):
        for j in range(len(A), 2*len(A)):
            if(j-len(A)==i):
                augmatrix[i][j] = 1
            else:
                augmatrix[i][j] = 0
    return augmatrix

def Inverse(A):
    newA = AugmentMatrixInverse(A)
    n = len(newA)
    NROW = np.zeros(n)
    pfound = False
    for i in range(0,n):
        NROW[i] = i
    for i in range(0, n-1):
        p = i
        maxim = 0
        for j in range(i, n):
            if(pfound == False):
                if(np.absolute(newA[int(NROW[j])][i])>maxim):
                    maxim = np.absolute(newA[int(NROW[j])][i])
                if(np.absolute(newA[int(NROW[p])][i])==maxim):
                    p = j
                    pfound = True
        if(newA[int(NROW[p])][i]==0):
            print('matrix is singular')
        
        if(NROW[i] != NROW[p]):
            temp = NROW[i]
            NROW[i] = NROW[p]
            NROW[p] = temp
        m = np.zeros((n,n+1))
        for j in range(i+1, n):
            m[int(NROW[j])][i]=newA[int(NROW[j])][i]/newA[int(NROW[i])][i]
            newA[int(NROW[j])]=newA[int(NROW[j])]- (m[int(NROW[j])][i]*newA[int(NROW[i])])
                     
    if(newA[int(NROW[n-1])][n-1]==0):
        print('matrix is singular')
        
    for i in reversed(range(0,n)):
        for j in reversed(range(0,i+1)):
            #print(i, ' ', j)
            if j == i:
                newA[int(NROW[i])] = newA[int(NROW[i])]/newA[int(NROW[i])][j]
                largesti = int(NROW[i])
                for k in reversed(range(0,i)):
                    newA[int(NROW[k])] = newA[int(NROW[k])] - newA[int(NROW[k])][j]*newA[largesti]
            else:
                newA[int(NROW[i])] = newA[int(NROW[i])] - newA[int(NROW[i])][j]*newA[largesti]
        #print(A1)
    Inverse = np.zeros((n,n))
    for i in range(0,n):
        for j in range(0,n):
            Inverse[i][j] = newA[int(NROW[i])][j+n]
    return Inverse

def InfinityNorm(a):
    max = np.absolute(a[0])
    for i in range(0, int(len(a))):
        if np.absolute(a[i]) > max:
            max = np.absolute(a[i])
    return max

def randmat(n):
    matrix = np.random.rand(n,n)
    for i in range (0,n):
        for j in range(0, n):
            if i == j:
                matrix[i][j] = n + matrix[i][j]
    return matrix

def randvec(n):
    vector = 100*np.random.rand(n)
    return vector


def GaussElim(A,b):
    if len(A) == len(b):
        newA = np.column_stack((A,b))
        n = len(A)
        NROW = np.zeros(n)
        pfound = False
        for i in range(0,n):
            NROW[i] = i
        for i in range(0, n-1):
            p=i
            maxim = 0
            for j in range(i, n):
                if(pfound == False):
                    if(np.absolute(newA[int(NROW[j])][i])>maxim):
                        maxim = np.absolute(newA[int(NROW[j])][i])
                    if(np.absolute(newA[int(NROW[p])][i])==maxim):
                        p = j
                        pfound = True
            if(newA[int(NROW[p])][i]==0):
                print('no unique solution exists')

            if(NROW[i] != NROW[p]):
                temp = NROW[i]
                NROW[i] = NROW[p]
                NROW[p] = temp
            m = np.zeros((n,n+1))
            for j in range(i+1, n):
                m[int(NROW[j])][i]=newA[int(NROW[j])][i]/newA[int(NROW[i])][i]
                newA[int(NROW[j])]=newA[int(NROW[j])]- (m[int(NROW[j])][i]*newA[int(NROW[i])])
      #  print(newA)
        if(newA[int(NROW[n-1])][n-1]==0):
            print('no unique solution exists')

        x = np.zeros(n)
        x[n-1] = newA[int(NROW[n-1])][n]/newA[int(NROW[n-1])][n-1]
        for i in reversed(range(0,n-1)):
            differand = 0
            for j in range(i+1, n):
                differand = differand + newA[int(NROW[i])][j]*x[j]
            x[i] = (newA[int(NROW[i])][n] - differand)/newA[int(NROW[i])][i]    
        return x
    else:
        print('invalid system')

def jacobi(A, b, err):
    n = len(A)
    D = np.zeros((n, n))
    L = np.zeros((n, n))
    U = np.zeros((n, n))
    for i in range(0, n):
        for j in range(0, n):
            if (j > i):
                U[i][j] = -1 * A[i][j]
            elif (j < i):
                L[i][j] = -1 * A[i][j]
            else:
                D[i][j] = A[i][j]
    #printmatrix(D)
    #printmatrix(U)
    #printmatrix(L)
    x = np.zeros(n)
    lusum = L + U
    # WE NEED TO WRITE OUR OWN MATRIX INVERSION AND MATRIX MULTIPLICATION
    coeffxk = MultiplyMatrix(Inverse(D), lusum)
    #printmatrix(lusum)
    endarray = MultiplyMatrixVector(Inverse(D), b)
    xprev = np.zeros(n)
    # printmatrix(coeffxk)
    count = 0
    moon = 0
    a = 1
    while(count < 20 and a > err):
        xkterm = MultiplyMatrixVector(coeffxk, x)
        x = xkterm + endarray
        #print("iteration ", count + 1, ": x = ", x)
        # print("rotation: ", count, "  x: ", x, "     xprev: ", xprev, "\n")
        a = InfinityNorm(x - xprev)
        #print("Error for iteration #", count+1, ": ", a)
        xprev = x
        count += 1
    return x


def GaussSeidel(A, b, err):
    n = len(A)
    D = np.zeros((n, n))
    L = np.zeros((n, n))
    U = np.zeros((n, n))
    for i in range (0, n):
        for j in range(0, n):
            if (j > i):
                U[i][j] = -1*A[i][j]
            elif (j < i):
                L[i][j] = -1*A[i][j]
            else:
                D[i][j] = A[i][j]
   # printmatrix(D)
   # printmatrix(U)
   # printmatrix(L)
    x = np.zeros(n)
    dldiff = D - L
    #WE NEED TO WRITE OUR OWN MATRIX INVERSION
    coeffxk = MultiplyMatrix(Inverse(dldiff),U)
     #printmatrix(lusum)
    endarray = MultiplyMatrixVector(Inverse(dldiff), b)
        #printmatrix(coeffxk)
    xprev = np.zeros(n)
    # printmatrix(coeffxk)
    count = 0
    moon = 0
    a = 1
    while(count <= 20 and 0.001< a):
        xkterm = MultiplyMatrixVector(coeffxk, x)
        x = xkterm + endarray
        #print("iteration ", count + 1, ": x = ", x)
        # print("rotation: ", count, "  x: ", x, "     xprev: ", xprev, "\n")
        a = InfinityNorm(x - xprev)
        #print("Error for iteration #", count+1, ": ", a)
        xprev = x
        count += 1

    return x


err = 0.000001


#x = np.linspace(100, 1000, 100)
#y = np.linspace(0, 10, 100)
#fig = plt.figure()
#plt.plot(x,y)  
#plt.title('Time Comparison')



for n in range(100, 1000, 100): 
    A = randmat(n)
    b = randvec(n)
    # test the time
    start = time.time()
    GaussElim(A, b)
    end = time.time()
    print(end - start)
    #start = time.time()
    #jacobi(A, b, err)
    #end = time.time()
    #print(end - start)
    #start = time.time()
    #GaussSeidel(A, b, err)
    #end = time.time()
    #print("GaussSeidel:\n",end - start)
    #plt.plot(y, 'g')
    




If you are interested in, then you can make a code using the "theoretically simplest method". For $Ax = b$, $x = A^{-1}b$. By using Gauss Elimination, you may compute $A^{-1}$ and then compute $A^{-1}b$. Recall that one can compute $A^{-1}$ as the following:
<ul>
    <li>Make an augmented matrix $[A | I]$ where $I$ is the $n \times n$ identity matrix.</li>
    <li>Apply elementary row operations until the left half $A$ on $[A| I]$ becomes $I$.</li>
    <li>Then the right half of the augmented matrix is $A^{-1}$.</li>
</ul>
Compare the performance of this method with above three methods.